# 🏋️‍♂️ Multi-Agent RAG for Fitness & Health with Azure AI Foundry 🥑

Welcome to this fun, self-guided workshop where we'll build a multi-agent Retrieval-Augmented Generation (RAG) pipeline using AutoGen 0.4.7 with Azure AI Foundry. Our team of agents will collaborate to answer your fitness and health questions in an engaging way!

## 1. Setup

Let's import the necessary libraries and set up our model client using Azure AI Foundry. Make sure your environment variable `GITHUB_TOKEN` is set with your personal access token.

In [3]:
pip install --upgrade pip
pip install python-dotenv
pip install pyautogen


Found existing installation: pyautogen 0.7.6
Uninstalling pyautogen-0.7.6:
  Successfully uninstalled pyautogen-0.7.6


In [1]:
# Import required Azure libraries
import os
import json  # For JSON operations
from pathlib import Path


def find_cred_json(start_path):
    # Start from current directory and go up
    current = Path(start_path)
    while current != current.parent:  # while we haven't hit the root
        cred_file = current / 'cred.json'
        if cred_file.exists():
            return str(cred_file)
        current = current.parent
    return None

try:
    # Search in the parent directory and its subdirectories
    parent_dir = os.path.dirname(os.getcwd())  # Get parent directory
    file_path = find_cred_json(parent_dir)

    if not file_path:
        raise FileNotFoundError("cred.json not found in parent directories")

    print(f"Found cred.json at: {file_path}")

    # Load and parse the JSON file
    with open(file_path, 'r') as f:
        loaded_config = json.load(f)

    
except Exception as e:
    print(f"❌ Error creating search clients: {e}")

Found cred.json at: D:\MLOps\Gen Ai & MLOps Masterclass\Materilas\test\ai-foundry-workshop\cred.json


In [2]:
import os
import asyncio

# Import AutoGen agents and required modules
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken

# Import the Azure AI Foundry model client from AutoGen extensions
from autogen_ext.models.azure import AzureAIChatCompletionClient
from autogen_core.models import UserMessage
from azure.core.credentials import AzureKeyCredential

# Create the model client using Azure AI Foundry
model_client = AzureAIChatCompletionClient(
    model=loaded_config.get("MODEL_DEPLOYMENT_NAME"),
    endpoint="https://models.inference.ai.azure.com",
    credential=AzureKeyCredential(loaded_config.get("GITHUB_TOKEN")),
    model_info={
        "json_output": False,
        "function_calling": False,
        "vision": False,
        "family": "unknown"
    }
)

print("✅ Azure AI Foundry model client created successfully!")

ModuleNotFoundError: No module named 'autogen_agentchat'

## 2. Create Sample Health Data and Retrieval Tool

We'll define a small list of health tips and a simple retrieval function. This function simulates retrieving relevant health tips based on keywords in the user's query.

In [ ]:
# Define sample health tips
health_tips = [
    {"id": "tip1", "content": "Do a 10-minute HIIT workout to boost your metabolism.", "source": "Fitness Guru"},
    {"id": "tip2", "content": "Take a brisk 15-minute walk to clear your mind and improve circulation.", "source": "Health Coach"},
    {"id": "tip3", "content": "Stretch for 5 minutes every hour if you're sitting at a desk.", "source": "Wellness Expert"},
    {"id": "tip4", "content": "Incorporate strength training twice a week for overall fitness.", "source": "Personal Trainer"},
    {"id": "tip5", "content": "Drink water regularly to stay hydrated during workouts.", "source": "Nutritionist"}
]

def retrieve_tips(query: str) -> str:
    """Return health tips whose content contains keywords from the query."""
    query_lower = query.lower()
    relevant = []
    for tip in health_tips:
        # Check if any word in the query is in the tip content
        if any(word in tip["content"].lower() for word in query_lower.split()):
            relevant.append(f"Source: {tip['source']} => {tip['content']}")
    if not relevant:
        # If no tips match, return all tips (for demo purposes)
        relevant = [f"Source: {tip['source']} => {tip['content']}" for tip in health_tips]
    return "\n".join(relevant)

print("✅ Sample health tips and retrieval tool created!")

## 3. Define Our Multi-Agent RAG Pipeline

We will create two agents:

1. **RetrieverAgent**: Uses the `retrieve_tips` tool to fetch relevant fitness and health tips based on the user's query.
2. **ResponderAgent**: Crafts a fun, engaging response using the retrieved tips along with the user's query.

These agents will be combined in a RoundRobinGroupChat to simulate collaborative problem-solving.

In [ ]:
# Define the RetrieverAgent with the retrieve_tips tool
retriever = AssistantAgent(
    name="RetrieverAgent",
    system_message=(
        "You are a smart retrieval agent. Your task is to fetch relevant fitness and health tips based on the user's query. "
        "Use the provided tool 'retrieve_tips' to get the information."
    ),
    model_client=model_client,
    tools=[retrieve_tips],  # Register the retrieval tool
    reflect_on_tool_use=True
)

# Define the ResponderAgent which uses the retrieved tips to craft a response
responder = AssistantAgent(
    name="ResponderAgent",
    system_message=(
        "You are a friendly fitness coach. Use the health tips provided to craft a fun and engaging response to the user's question."
    ),
    model_client=model_client
)

# Combine the two agents in a RoundRobinGroupChat
group_chat = RoundRobinGroupChat(
    agents=[retriever, responder],
    termination_condition=None,  # For simplicity, we run a fixed number of turns
    max_turns=4  # Limit the conversation to 4 turns
)

print("✅ Multi-agent RAG pipeline defined!")

## 4. Try a Query

Let's test our multi-agent RAG system with a fun fitness query. For example:

> **User Query:** _I'm very busy but want to stay fit. What quick exercises can I do?_

In [ ]:
async def run_query():
    user_query = "I'm very busy but want to stay fit. What quick exercises can I do?"

    # Run the group chat to process the query
    final_result = None
    async for message in group_chat.run_stream(task=user_query):
        # Print each message as it is produced
        if hasattr(message, 'content'):
            print(f"{message.source}: {message.content}")
        final_result = message
    
    print("\n✅ Final response:", final_result.content if final_result else "No response")

asyncio.run(run_query())

## 5. Conclusion

In this notebook, we built a fun multi-agent Retrieval-Augmented Generation pipeline using AutoGen 0.4.7 with Azure AI Foundry and a fitness and health theme. We created a RetrieverAgent that fetches relevant health tips and a ResponderAgent that crafts an engaging answer to the user's query.

Feel free to modify and expand this notebook to explore more advanced multi-agent collaborations. Happy coding and stay fit! 💪🥦